In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10, 7)
plt.rcParams["font.size"] = 12

In [2]:
import inspect

import numpy as np
import xarray as xr
import xarray_sentinel

from sarsen import apps, geocoding, orbit, scene

# uncomment to check that the code below is in sync with the implementation
# print(inspect.getsource(apps.backward_geocode_sentinel1))

# define input and load data

In [3]:
product_urlpath = (
    "data/S1B_IW_SLC__1SDV_20211223T051121_20211223T051148_030148_039993_BA4B.SAFE/"
)
measurement_group = "IW3/VV"
dem_urlpath = "data/Rome-10m-DEM.tif"
orbit_group = None
calibration_group = None
output_urlpath = "Rome-10m-GTC-SLC.tif"
correct_radiometry = False
interp_method = "nearest"
multilook = None
grouping_area_factor = (1.0, 1.0)
open_dem_raster_kwargs = {"chunks": {}}
kwargs = {"chunks": {"pixel": 2048}}

In [4]:
!ls -d {product_urlpath}
!ls -d {dem_urlpath}

data/S1B_IW_SLC__1SDV_20211223T051121_20211223T051148_030148_039993_BA4B.SAFE/
data/Rome-10m-DEM.tif


In [5]:
orbit_group = orbit_group or f"{measurement_group}/orbit"
calibration_group = calibration_group or f"{measurement_group}/calibration"

measurement_ds = xr.open_dataset(product_urlpath, engine="sentinel-1", group=measurement_group, **kwargs)  # type: ignore
measurement = measurement_ds.measurement

dem_raster = scene.open_dem_raster(dem_urlpath, **open_dem_raster_kwargs)

orbit_ecef = xr.open_dataset(product_urlpath, engine="sentinel-1", group=orbit_group, **kwargs)  # type: ignore
position_ecef = orbit_ecef.position
calibration = xr.open_dataset(product_urlpath, engine="sentinel-1", group=calibration_group, **kwargs)  # type: ignore
beta_nought_lut = calibration.betaNought

# scene

In [6]:
dem_raster

<xarray.DataArray 'band_data' (y: 5000, x: 5000)>
dask.array<getitem, shape=(5000, 5000), dtype=float32, chunksize=(5000, 5000), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 2.75e+05 2.75e+05 2.75e+05 ... 3.25e+05 3.25e+05
  * y            (y) float64 4.625e+06 4.625e+06 ... 4.675e+06 4.675e+06
    spatial_ref  int64 ...

In [7]:
# _ = dem_raster.plot()

In [8]:
%%time
dem_ecef = scene.convert_to_dem_ecef(dem_raster)
dem_ecef

CPU times: user 12.6 s, sys: 1.14 s, total: 13.8 s
Wall time: 13.5 s


<xarray.DataArray 'dem_3d' (axis: 3, y: 5000, x: 5000)>
dask.array<setitem, shape=(3, 5000, 5000), dtype=float64, chunksize=(3, 2500, 2500), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 2.75e+05 2.75e+05 2.75e+05 ... 3.25e+05 3.25e+05
    spatial_ref  int64 ...
  * y            (y) float64 4.625e+06 4.625e+06 ... 4.675e+06 4.675e+06
  * axis         (axis) int64 0 1 2

# acquisition

In [9]:
measurement

<xarray.DataArray 'measurement' (line: 13671, pixel: 25061)>
dask.array<open_dataset-0a8821f5cd97ff90d59ecc772670c5a9measurement, shape=(13671, 25061), dtype=complex64, chunksize=(1519, 2048), chunktype=numpy.ndarray>
Coordinates:
  * pixel             (pixel) int64 0 1 2 3 4 ... 25056 25057 25058 25059 25060
  * line              (line) int64 0 1 2 3 4 5 ... 13666 13667 13668 13669 13670
    azimuth_time      (line) datetime64[ns] dask.array<chunksize=(13671,), meta=np.ndarray>
    slant_range_time  (pixel) float64 dask.array<chunksize=(2048,), meta=np.ndarray>
Attributes:
    sar:center_frequency:       5.40500045433435
    sar:pixel_spacing_azimuth:  13.88904
    sar:pixel_spacing_range:    2.329562
    azimuth_time_interval:      0.002055556299999998
    slant_range_time_interval:  1.554116558005821e-08
    sat:anx_datetime:           2021-12-23T04:33:35.556206Z
    azimuth_steering_rate:      1.397440818
    number_of_bursts:           9
    lines_per_burst:            1519
    units:                      1
    long_name:                  digital number

In [10]:
%%time
acquisition = apps.simulate_acquisition(position_ecef, dem_ecef)
acquisition

CPU times: user 50.8 s, sys: 18.6 s, total: 1min 9s
Wall time: 14.4 s


<xarray.Dataset>
Dimensions:           (x: 5000, y: 5000, axis: 3)
Coordinates:
  * x                 (x) float64 2.75e+05 2.75e+05 ... 3.25e+05 3.25e+05
    spatial_ref       int64 0
  * y                 (y) float64 4.625e+06 4.625e+06 ... 4.675e+06 4.675e+06
  * axis              (axis) int64 0 1 2
Data variables:
    azimuth_time      (y, x) datetime64[ns] dask.array<chunksize=(2500, 2500), meta=np.ndarray>
    slant_range_time  (y, x) float64 dask.array<chunksize=(2500, 2500), meta=np.ndarray>
    dem_direction     (axis, y, x) float64 dask.array<chunksize=(3, 2500, 2500), meta=np.ndarray>

In [11]:
%%time
beta_nought = xarray_sentinel.calibrate_intensity(measurement, beta_nought_lut)
beta_nought

CPU times: user 6.24 ms, sys: 998 µs, total: 7.24 ms
Wall time: 6.67 ms


<xarray.DataArray (line: 13671, pixel: 25061)>
dask.array<pow, shape=(13671, 25061), dtype=float32, chunksize=(1519, 2048), chunktype=numpy.ndarray>
Coordinates:
  * pixel             (pixel) int64 0 1 2 3 4 ... 25056 25057 25058 25059 25060
  * line              (line) int64 0 1 2 3 4 5 ... 13666 13667 13668 13669 13670
    azimuth_time      (line) datetime64[ns] dask.array<chunksize=(13671,), meta=np.ndarray>
    slant_range_time  (pixel) float64 dask.array<chunksize=(2048,), meta=np.ndarray>
Attributes:
    sar:center_frequency:       5.40500045433435
    sar:pixel_spacing_azimuth:  13.88904
    sar:pixel_spacing_range:    2.329562
    azimuth_time_interval:      0.002055556299999998
    slant_range_time_interval:  1.554116558005821e-08
    sat:anx_datetime:           2021-12-23T04:33:35.556206Z
    azimuth_steering_rate:      1.397440818
    number_of_bursts:           9
    lines_per_burst:            1519
    units:                      m2 m-2
    long_name:                  beta nought

In [12]:
%%time
coordinate_conversion = None
if measurement_ds.attrs["sar:product_type"] == "GRD":
    coordinate_conversion = xr.open_dataset(
        product_urlpath,
        engine="sentinel-1",
        group=f"{measurement_group}/coordinate_conversion",
        **kwargs,
    )  # type: ignore
    ground_range = xarray_sentinel.slant_range_time_to_ground_range(
        acquisition.azimuth_time,
        acquisition.slant_range_time,
        coordinate_conversion,
    )
    interp_kwargs = {"ground_range": ground_range}
elif measurement_ds.attrs["sar:product_type"] == "SLC":
    interp_kwargs = {"slant_range_time": acquisition.slant_range_time}
    if measurement_ds.attrs["sar:instrument_mode"] == "IW":
        beta_nought = xarray_sentinel.mosaic_slc_iw(beta_nought)
else:
    raise ValueError(
        f"unsupported sar:product_type {measurement_ds.attrs['sar:product_type']}"
    )

CPU times: user 29.7 ms, sys: 2 ms, total: 31.7 ms
Wall time: 31.3 ms


In [13]:
%%time
geocoded = apps.interpolate_measurement(
    beta_nought,
    multilook=multilook,
    azimuth_time=acquisition.azimuth_time,
    interp_method=interp_method,
    **interp_kwargs,
)

geocoded

CPU times: user 14.1 s, sys: 3.31 s, total: 17.4 s
Wall time: 8.1 s


<xarray.DataArray (y: 5000, x: 5000)>
dask.array<dask_aware_interpnd, shape=(5000, 5000), dtype=float32, chunksize=(2500, 2500), chunktype=numpy.ndarray>
Coordinates:
    pixel             (y, x) float64 1.78e+04 1.78e+04 ... 5.915e+03 5.912e+03
    line              (y, x) float64 9.541e+03 9.541e+03 ... 4.795e+03 4.795e+03
    azimuth_time      (y, x) datetime64[ns] dask.array<chunksize=(2500, 2500), meta=np.ndarray>
    slant_range_time  (y, x) float64 dask.array<chunksize=(2500, 2500), meta=np.ndarray>
  * x                 (x) float64 2.75e+05 2.75e+05 ... 3.25e+05 3.25e+05
    spatial_ref       int64 0
  * y                 (y) float64 4.625e+06 4.625e+06 ... 4.675e+06 4.675e+06
Attributes:
    sar:center_frequency:       5.40500045433435
    sar:pixel_spacing_azimuth:  13.88904
    sar:pixel_spacing_range:    2.329562
    azimuth_time_interval:      0.002055556299999998
    slant_range_time_interval:  1.554116558005821e-08
    sat:anx_datetime:           2021-12-23T04:33:35.556206Z
    azimuth_steering_rate:      1.397440818
    number_of_bursts:           9
    lines_per_burst:            1519
    units:                      m2 m-2
    long_name:                  beta nought 
    azimuth_anx_time:           2266.249436
    burst_index:                0

In [14]:
geocoded.rio.set_crs(dem_raster.rio.crs)
geocoded.rio.to_raster(
    output_urlpath,
    dtype=np.float32,
    tiled=True,
    blockxsize=512,
    blockysize=512,
    compress="ZSTD",
    num_threads="ALL_CPUS",
)

In [15]:
# _ = geocoded.plot(vmax=1.0)